In [2]:
!pip install transformers

In [37]:
import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
import nltk
from nltk.tokenize import word_tokenize
import re
nltk.download('punkt_tab')
nltk.download('punkt')
context = """
Natural Language Processing (NLP) is a specialized branch within the broader field of Artificial Intelligence (AI) that focuses on enabling machines to understand, interpret, generate, and respond to human language in a meaningful way. It sits at the intersection of computer science, linguistics, and cognitive psychology, combining techniques from these disciplines to allow computers to process natural language data both in written and spoken forms. The goal of NLP is not just to parse language as a string of characters or words, but to truly grasp its underlying meaning, intent, and context—mimicking, as closely as possible, the human ability to communicate.

At the core of NLP are several foundational techniques such as tokenization, part-of-speech tagging, named entity recognition (NER), syntactic and semantic parsing, word embeddings, and more recently, context-aware vector representations through transformer models. These techniques are powered by a mix of rule-based systems, statistical models, deep learning architectures, and large-scale pretraining on text corpora.

NLP enables a wide array of real-world applications that we often interact with daily. These include machine translation systems like Google Translate, speech recognition systems such as Apple’s Siri or Amazon Alexa, sentiment analysis engines used in social media monitoring, automatic summarization tools that condense large documents into short abstracts, and chatbots that serve customers across sectors. Another prominent use case is question answering (QA), where a system processes a natural language query and returns a precise answer drawn from a given context or dataset.

In recent years, the field has been revolutionized by the advent of large language models (LLMs), such as BERT (Bidirectional Encoder Representations from Transformers), GPT (Generative Pre-trained Transformer), T5 (Text-to-Text Transfer Transformer), and others. These models are trained on massive datasets and are capable of capturing nuanced linguistic patterns and semantics. BERT is particularly well-suited for tasks that require deep understanding of sentence context, as it reads text bidirectionally. GPT, on the other hand, focuses on generating coherent, fluent text and is optimized for text completion and open-ended dialogue tasks. T5 generalizes all NLP tasks into a text-to-text format, making it a versatile tool for both classification and generation.

Among the most widely deployed NLP technologies are chatbots. These systems can be categorized into two main types: rule-based chatbots, which rely on manually defined patterns and logic trees; and AI-driven chatbots, which use machine learning and neural networks to learn from data and generate more natural, human-like interactions. AI-driven chatbots are capable of understanding user intent, maintaining the flow of dialogue over multiple turns, adapting to different contexts, and offering personalized responses based on user history and behavior. They are employed in domains ranging from customer service and banking to healthcare, education, and virtual assistants.

Building an effective QA chatbot is a complex task that goes beyond simply extracting sentences from documents. The system must be able to comprehend a given passage, identify the relevant information, and return concise, correct answers. This requires not only robust language models but also high-quality datasets for training and benchmarking. Datasets like SQuAD (Stanford Question Answering Dataset), Natural Questions (by Google), and HotpotQA (which includes multi-hop reasoning) have played a crucial role in advancing QA research by providing standard benchmarks and evaluation protocols.

Moreover, QA chatbots must deal with several challenges in real-world applications. These include understanding ambiguous questions, resolving pronouns and references, handling multi-turn conversations, and dealing with the noise and variability in user input. To address these challenges, researchers are increasingly working on context-aware QA systems, which maintain memory of previous questions and answers, use coreference resolution techniques, and incorporate dialogue management strategies.

As NLP continues to evolve rapidly, so do the capabilities of QA systems and conversational agents. New directions in research include the development of multilingual and cross-lingual QA systems, which can answer questions in multiple languages; zero-shot and few-shot learning, where models generalize from very few examples; and real-time QA, where answers are generated or retrieved instantly from dynamic data sources. These advancements are pushing the boundaries of what is possible with language technology and are paving the way for more intelligent, responsive, and human-like interactions between people and machines.
"""

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [44]:
def resolve_pronouns(question, memory):
    if not memory:
        return question

    pronouns = ["it", "its", "they", "them", "their", "this", "that", "these", "those"]

    words = word_tokenize(question.lower())
    contains_pronoun = any(word in pronouns for word in words)

    if contains_pronoun:
        recent_interactions = memory[-3:][::-1]

        entity_candidates = []

        for interaction in recent_interactions:
            prev_question = interaction["processed_question"]
            prev_answer = interaction["answer"]

            stopwords = set(pronouns + ["what", "when", "where", "who", "how", "why",
                                      "is", "are", "was", "were", "will", "would",
                                      "should", "could", "did", "do", "does", "a",
                                      "an", "the", "in", "on", "at", "by", "for"])

            for text in [prev_question, prev_answer]:
                tokens = word_tokenize(text)
                for i in range(len(tokens)):
                    if tokens[i].lower() not in stopwords and len(tokens[i]) > 2:
                        entity_candidates.append(tokens[i])

                        if i < len(tokens) - 1 and tokens[i+1].lower() not in stopwords:
                            entity_candidates.append(f"{tokens[i]} {tokens[i+1]}")

                            if i < len(tokens) - 2 and tokens[i+2].lower() not in stopwords:
                                entity_candidates.append(f"{tokens[i]} {tokens[i+1]} {tokens[i+2]}")

        entity_candidates = sorted(set(entity_candidates), key=len, reverse=True)

        if entity_candidates:
            words = question.split()
            for i, word in enumerate(words):
                clean_word = re.sub(r'[^\w\s]', '', word.lower())
                if clean_word in pronouns:
                    best_candidate = entity_candidates[0]

                    if word[0].isupper():
                        best_candidate = best_candidate[0].upper() + best_candidate[1:]

                    punctuation = re.sub(r'[\w\s]', '', word)
                    if punctuation:
                        best_candidate += punctuation

                    words[i] = best_candidate
                    break

            processed_question = " ".join(words)
            print(f"Resolved question: {processed_question}")
            return processed_question

    return question

In [45]:
def answer_question(question, context, qa_pipeline=None):
    if qa_pipeline is None:
        model_name = "distilbert-base-cased-distilled-squad"
        qa_pipeline = pipeline("question-answering", model=model_name)

    result = qa_pipeline(question=question, context=context)

    return result["answer"]

In [46]:
def run_chatbot(context_file_path=None, context_text=None):
    model_name = "distilbert-base-cased-distilled-squad"
    qa_pipeline = pipeline("question-answering", model=model_name)

    memory = []

    print("NLP QA Chatbot (type 'exit' to quit)")

    while True:
        user_input = input("\nYou: ").strip()

        if user_input.lower() == 'exit':
            print("Goodbye")
            break


        processed_question = resolve_pronouns(user_input, memory)

        if processed_question != user_input:
            print(f"(Resolving pronouns: '{user_input}' → '{processed_question}')")

        answer = answer_question(processed_question, context, qa_pipeline)

        memory.append({
            "question": user_input,
            "processed_question": processed_question,
            "answer": answer
        })

        if len(memory) > 5:
            memory.pop(0)

        print(f"\nBot: {answer}")


In [47]:
run_chatbot(context_text=context)

Device set to use cpu


NLP QA Chatbot (type 'exit' to quit)

You: what is NLP?

Bot: Natural Language Processing

You: what does it do?
Resolved question: what does Natural Language Processing do?
(Resolving pronouns: 'what does it do?' → 'what does Natural Language Processing do?')

Bot: returns a precise answer drawn from a given context or dataset

You: give examples of NLP applications

Bot: machine translation systems like Google Translate

You: what about TS?

Bot: standard benchmarks and evaluation protocols

You: what about GPT?

Bot: Generative Pre-trained Transformer

You: what can they do?
Resolved question: what can Generative Pre-trained Transformer do?
(Resolving pronouns: 'what can they do?' → 'what can Generative Pre-trained Transformer do?')

Bot: GPT

You: exit
Goodbye
